In [1]:
import cv2
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Histogram Feature Extraction

In [ ]:
import cv2
import numpy as np
import os

# Function to compute the histogram for each channel manually
def compute_histogram(patch, bins=8):
    hist_vector = np.zeros(bins)
    bin_width = 256 // bins

    # Flatten the patch for simpler looping
    patch_flat = patch.flatten()

    # Count occurrences of pixel values in each bin
    for pixel_value in patch_flat:
        bin_index = pixel_value // bin_width
        hist_vector[bin_index] += 1

    # Normalize the histogram by the total number of pixels in the patch
    hist_vector /= len(patch_flat)
    return hist_vector

# Function to extract a 24-dim color histogram from a patch
def color_histogram(patch, bins=8):
    hist_r = compute_histogram(patch[:, :, 0], bins=bins)
    hist_g = compute_histogram(patch[:, :, 1], bins=bins)
    hist_b = compute_histogram(patch[:, :, 2], bins=bins)

    # Concatenate histograms to form a 24-dim vector
    hist_vector = np.concatenate([hist_r, hist_g, hist_b])
    return hist_vector

# Function to extract patches and their histograms from an image
def extract_histogram_features(image, patch_size=32):
    h, w, _ = image.shape
    features = []

    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            if patch.shape[:2] == (patch_size, patch_size):  # Ensure patch size matches
                hist_vector = color_histogram(patch)
                features.append(hist_vector)

    return np.array(features)  # Stack all patch histograms for the image

# Function to process all images in a folder and save their features
def process_image_folder(image_folder, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)

        # Extract 24-dimensional histogram features
        features = extract_histogram_features(image)

        # Save features in the corresponding folder
        save_path = os.path.join(save_folder, f"{os.path.splitext(image_name)[0]}_features.npy")
        np.save(save_path, features)

# Example usage
# class_folders = ['class1', 'class2', 'class3']
# for class_folder in class_folders:
#     process_image_folder(f"./images/{class_folder}", f"./features/{class_folder}")


#process_image_folder(f"/content/drive/MyDrive/group07(3)/group07/train/",f"/content/drive/MyDrive/group07(3)/group07/arch_new")


In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/train/arch/"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features/arch_new/")
else:
    print(f"Directory does not exist: {image_folder}")


KeyboardInterrupt: 

In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/test/arch/"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features_test/arch_new/")
else:
    print(f"Directory does not exist: {image_folder}")

In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/train/forest_path"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features/forest_path_new/")
else:
    print(f"Directory does not exist: {image_folder}")


In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/test/forest_path"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features_test/forest_path_new/")
else:
    print(f"Directory does not exist: {image_folder}")


In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/train/highway/"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features/highway_new/")
else:
    print(f"Directory does not exist: {image_folder}")


In [ ]:
image_folder = "/content/drive/MyDrive/group07_scene/group07/test/highway/"
if os.path.exists(image_folder):
    process_image_folder(image_folder, "/content/drive/MyDrive/group07/features_test/highway_new/")
else:
    print(f"Directory does not exist: {image_folder}")

## Clustering in 32 clusters

In [2]:
import numpy as np

# Function to compute Euclidean distance between two points
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# K-means clustering algorithm implementation from scratch
def kmeans_clustering(data, k, max_iters=3):
    # Randomly initialize cluster centroids
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]

    # Initialize cluster labels for each point
    labels = np.zeros(data.shape[0])

    for iteration in range(max_iters):
        # Step 1: Assign each point to the nearest centroid
        for i in range(data.shape[0]):
            distances = np.array([euclidean_distance(data[i], centroid) for centroid in centroids])
            labels[i] = np.argmin(distances)

        # Step 2: Update the centroids by calculating the mean of the points in each cluster
        new_centroids = np.zeros(centroids.shape)
        for i in range(k):
            points_in_cluster = data[labels == i]
            if len(points_in_cluster) > 0:
                new_centroids[i] = np.mean(points_in_cluster, axis=0)

        # If the centroids don't change, we have converged
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids

    return centroids, labels

In [3]:
def collect_feature_vectors(feature_folder):
    all_features = []
    for class_folder in os.listdir(feature_folder):
        class_path = os.path.join(feature_folder, class_folder)
        for feature_file in os.listdir(class_path):
            feature_path = os.path.join(class_path, feature_file)
            features = np.load(feature_path)
            all_features.append(features)

    return np.vstack(all_features)  # Combine all features into one array

# Example usage of kmeans
train_features = collect_feature_vectors("/content/drive/MyDrive/group07/features/")
k = 32
centroids, labels = kmeans_clustering(train_features, k)
print(len(centroids))
print(len(labels))

32
194877


In [4]:
test_features = collect_feature_vectors("/content/drive/MyDrive/group07/features_test/")
k = 32
centroids_test, labels_test = kmeans_clustering(test_features, k)
print(len(centroids_test))
print(len(labels_test))

32
169332


##BOVW Features Extraction



In [5]:
# Function to assign histogram vectors to clusters and form BoVW representation
def bovw_representation(image_features, centroids):
    k = centroids.shape[0]
    bovw_vector = np.zeros(k)

    # Assign each feature to the closest centroid
    for feature in image_features:
        distances = np.array([euclidean_distance(feature, centroid) for centroid in centroids])
        nearest_centroid = np.argmin(distances)
        bovw_vector[nearest_centroid] += 1

    # Normalize the BoVW vector
    bovw_vector /= len(image_features)
    return bovw_vector

# Generate BoVW representations for all images in the test set
def generate_bovw_for_images(feature_folder, centroids, save_folder):
    bovw_features=[]
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    for class_folder in os.listdir(feature_folder):
        class_path = os.path.join(feature_folder, class_folder)
        for feature_file in os.listdir(class_path):
            feature_path = os.path.join(class_path, feature_file)
            features = np.load(feature_path)

            # Generate BoVW representation
            bovw_vector = bovw_representation(features, centroids)
            bovw_features.append(bovw_vector)

            # Save the BoVW vector
            save_path = os.path.join(save_folder, f"{os.path.splitext(feature_file)[0]}_bovw.npy")
            np.save(save_path, bovw_vector)
    return bovw_features


# Example usage
bovw_features_train=[]
bovw_features_train=generate_bovw_for_images("/content/drive/MyDrive/group07/features/", centroids, "/content/drive/MyDrive/group07/bovw")


In [6]:
len(bovw_features_train)

150

In [7]:
bovw_features_test=[]
bovw_features_test=generate_bovw_for_images("/content/drive/MyDrive/group07/features_test/", centroids_test, "/content/drive/MyDrive/group07/bovw_test")

In [8]:
len(bovw_features_test)

150

In [9]:
import os
import numpy as np

def collect_bovw_vectors(bovw_folder):
    bovw_vectors = []

    # Iterate through all files in the bovw folder
    for file_name in os.listdir(bovw_folder):
        if file_name.endswith(".npy"):  # Check if the file is an .npy file
            file_path = os.path.join(bovw_folder, file_name)

            # Load the BoVW vector and append it to the list
            bovw_vector = np.load(file_path)
            bovw_vectors.append(bovw_vector)

    return np.array(bovw_vectors)

# Example usage
# bovw_folder = "/content/drive/MyDrive/group07/features/bovw"
# bovw_vectors = collect_bovw_vectors(bovw_folder)

# Now bovw_vectors is a list of all BoVW vectors from the folder


## Classification using bovw features

In [41]:
# def gaussian_density(x, mean, cov):
#     n = len(x)
#     cov_inv = np.linalg.inv(cov)
#     norm_const = 1.0 / (np.power((2 * np.pi), n / 2) * np.sqrt(np.linalg.det(cov)))
#     x_mean = x - mean
#     return norm_const * np.exp(-0.5 * np.dot(np.dot(x_mean.T, cov_inv), x_mean))

def gaussian_density(x, mean, cov):
    n = len(x)
    # Add regularization to covariance matrix
    cov = cov + np.eye(cov.shape[0]) * 1e-3
    cov_inv = np.linalg.inv(cov)
    norm_const = 1.0 / (np.power((2 * np.pi), n / 2) * np.sqrt(np.linalg.det(cov)))
    x_mean = x - mean
    return norm_const * np.exp(-0.5 * (x_mean.T @ cov_inv @ x_mean))

In [42]:
def initialize_gmm_params(X, n_components):
    centroids, labels = kmeans_clustering(X, n_components)
    means = centroids
    covariances = np.array([np.cov(X[labels == i].T) for i in range(n_components)])
    priors = np.array([np.mean(labels == i) for i in range(n_components)])
    return means, covariances, priors

In [49]:
# def gmm_em(X, n_components, max_iter=5):
#     n_samples, n_features = X.shape
#     means, covariances, priors = initialize_gmm_params(X, n_components)

#     for _ in range(max_iter):
#         # E-step: Calculate responsibilities (gamma)
#         responsibilities = np.zeros((n_samples, n_components))
#         for i in range(n_components):
#             for j in range(n_samples):
#                 responsibilities[j, i] = priors[i] * gaussian_density(X[j], means[i], covariances[i])

#         responsibilities /= responsibilities.sum(axis=1, keepdims=True)

#         # M-step: Update parameters
#         N_k = responsibilities.sum(axis=0)
#         means = np.array([np.dot(responsibilities[:, k], X) / N_k[k] for k in range(n_components)])
#         covariances = np.array([np.dot((responsibilities[:, k] * (X - means[k]).T), (X - means[k])) / N_k[k] for k in range(n_components)])
#         priors = N_k / n_samples

#     return means, covariances, priors

def gmm_em(X, n_components, max_iter=5,reg=1e-6):
    n_samples, n_features = X.shape
    means, covariances, priors = initialize_gmm_params(X, n_components)

    # Calculate labels here to access within the function
    _, labels = kmeans_clustering(X, n_components)

    for _ in range(max_iter):
        # E-step: Calculate responsibilities (gamma)
        responsibilities = np.zeros((n_samples, n_components))
        for i in range(n_components):
            try:
                for j in range(n_samples):
                    responsibilities[j, i] = priors[i] * gaussian_density(X[j], means[i], covariances[i])
            except np.linalg.LinAlgError:
                print(f"Error in cluster {i}")
                print(f"Number of points in cluster {i}: {len(X[labels == i])}")
                raise

        responsibilities /= (responsibilities.sum(axis=1, keepdims=True)+1e-10)

        # M-step: Update parameters
        N_k = responsibilities.sum(axis=0)
        means = np.array([np.dot(responsibilities[:, k], X) / N_k[k] for k in range(n_components)])
        covariances = np.array([np.dot((responsibilities[:, k] * (X - means[k]).T), (X - means[k])) / N_k[k] for k in range(n_components)])
        for i in range(n_components):
            covariances[i] += reg * np.eye(n_features)
        priors = N_k / n_samples

    return means, covariances, priors

In [50]:


def classify(X_test, gmm_params_by_class):
    predictions = []
    for x in X_test:
        posteriors = []
        for class_label, (means, covariances, priors) in gmm_params_by_class.items():
            posterior = 0
            for i in range(len(means)):
                likelihood = gaussian_density(x, means[i], covariances[i])
                posterior += priors[i] * likelihood
            posteriors.append(posterior)
        predictions.append(np.argmax(posteriors))
    return np.array(predictions)

# Fit GMM for each class
def fit_gmm_for_all_classes(X_class1, X_class2, X_class3, n_components):
    gmm_params_by_class = {}

    # Class 1
    means1, covariances1, priors1 = gmm_em(X_class1, n_components)
    gmm_params_by_class[0] = (means1, covariances1, priors1)

    # Class 2
    means2, covariances2, priors2 = gmm_em(X_class2, n_components)
    gmm_params_by_class[1] = (means2, covariances2, priors2)

    # Class 3
    means3, covariances3, priors3 = gmm_em(X_class3, n_components)
    gmm_params_by_class[2] = (means3, covariances3, priors3)

    return gmm_params_by_class

In [51]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

def run_experiment(X_class1, X_class2, X_class3, X_test, y_test, n_mixtures_list):
    results = {}

    for n_mixtures in n_mixtures_list:
        # Fit GMM for all three classes
        gmm_params_by_class = fit_gmm_for_all_classes(X_class1, X_class2, X_class3, n_mixtures)

        # Predict on test data
        y_pred = classify(X_test, gmm_params_by_class)

        # Compute accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Generate classification report
        report = classification_report(y_test, y_pred, output_dict=True)

        # Generate confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Print out the results for this experiment
        print(f"GMM with {n_mixtures} mixtures:")
        print(f"  Accuracy = {accuracy:.4f}")
        print(classification_report(y_test, y_pred))  # Display full report
        print(f"Confusion Matrix:\n{cm}\n")

        # Store results for this number of mixtures
        results[n_mixtures] = {
            "accuracy": accuracy,
            "classification_report": report,
            "confusion_matrix": cm
        }

    return results


In [52]:
y_test = np.array([0] * 50 + [1] * 50 + [2] * 50)

In [53]:
bovw_features_train = np.array(bovw_features_train)
bovw_features_test = np.array(bovw_features_test)

In [54]:
n_mixtures_list = [1,2,4,8,16]
results = run_experiment(bovw_features_train[:50], bovw_features_train[50:100], bovw_features_train[100:], bovw_features_test, y_test, n_mixtures_list)

GMM with 1 mixtures:
  Accuracy = 0.2867
              precision    recall  f1-score   support

           0       0.30      0.56      0.39        50
           1       0.35      0.14      0.20        50
           2       0.22      0.16      0.18        50

    accuracy                           0.29       150
   macro avg       0.29      0.29      0.26       150
weighted avg       0.29      0.29      0.26       150

Confusion Matrix:
[[28  7 15]
 [29  7 14]
 [36  6  8]]

GMM with 2 mixtures:
  Accuracy = 0.3267
              precision    recall  f1-score   support

           0       0.33      0.76      0.46        50
           1       0.47      0.16      0.24        50
           2       0.17      0.06      0.09        50

    accuracy                           0.33       150
   macro avg       0.32      0.33      0.26       150
weighted avg       0.32      0.33      0.26       150

Confusion Matrix:
[[38  6  6]
 [33  8  9]
 [44  3  3]]

GMM with 4 mixtures:
  Accuracy = 0.2733
   

<ipython-input-42-eb9e87e8685a>:4: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariances = np.array([np.cov(X[labels == i].T) for i in range(n_components)])
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-pack

GMM with 8 mixtures:
  Accuracy = 0.3333
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50

    accuracy                           0.33       150
   macro avg       0.11      0.33      0.17       150
weighted avg       0.11      0.33      0.17       150

Confusion Matrix:
[[50  0  0]
 [50  0  0]
 [50  0  0]]



/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


GMM with 16 mixtures:
  Accuracy = 0.3333
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50

    accuracy                           0.33       150
   macro avg       0.11      0.33      0.17       150
weighted avg       0.11      0.33      0.17       150

Confusion Matrix:
[[50  0  0]
 [50  0  0]
 [50  0  0]]



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

##Classification using histogram features

In [ ]:
# def calculate_image_likelihood(image_features, gmm_params_by_class):
#     likelihoods_by_class = {}
#     for class_label, (means, covariances, priors) in gmm_params_by_class.items():
#         image_likelihood = 1.0
#         for patch_features in image_features:
#             patch_likelihood = 0
#             for i in range(len(means)):
#                 likelihood = gaussian_density(patch_features, means[i], covariances[i])
#                 patch_likelihood += priors[i] * likelihood
#             image_likelihood *= patch_likelihood
#         likelihoods_by_class[class_label] = image_likelihood
#     return likelihoods_by_class

def calculate_image_log_likelihood(image_features, gmm_params_by_class):
    log_likelihoods_by_class = {}
    for class_label, (means, covariances, priors) in gmm_params_by_class.items():
        image_log_likelihood = 0  # Initialize with 0 for summation
        for patch_features in image_features:
            patch_log_likelihood = -np.inf  # Initialize with -inf for max operation
            for i in range(len(means)):
                likelihood = gaussian_density(patch_features, means[i], covariances[i])

                # Calculate log-likelihood and handle potential zero likelihood
                log_likelihood_component = np.log(priors[i]) + np.log(likelihood) if likelihood > 0 else -np.inf

                patch_log_likelihood = max(patch_log_likelihood, log_likelihood_component)  # Use max for numerical stability

            image_log_likelihood += patch_log_likelihood  # Sum log-likelihoods of patches

        log_likelihoods_by_class[class_label] = image_log_likelihood
    return log_likelihoods_by_class

In [ ]:
def train_gmms(feature_folder, n_components=3):
    gmm_params_by_class = {}
    class_labels = {'arch_new': 0, 'forest_path_new': 1, 'highway_new': 2}

    for class_name, class_label in class_labels.items():
        class_path = os.path.join(feature_folder, class_name)
        class_features = []
        for feature_file in os.listdir(class_path):
            feature_path = os.path.join(class_path, feature_file)
            features = np.load(feature_path)
            class_features.extend(features)
        means, covariances, priors = gmm_em(np.array(class_features), n_components)
        gmm_params_by_class[class_label] = (means, covariances, priors)

    return gmm_params_by_class

In [ ]:
def classify_image(image_features, gmm_params_by_class):
    log_likelihoods = calculate_image_log_likelihood(image_features, gmm_params_by_class)
    predicted_class = max(log_likelihoods, key=log_likelihoods.get)
    return predicted_class

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
def run_experiment(feature_folder_train, feature_folder_test, n_components=3):
    # Train GMMs
    gmm_params_by_class = train_gmms(feature_folder_train, n_components)

    # Load test images and classify
    y_true = []
    y_pred = []
    class_labels = {'arch_new': 0, 'forest_path_new': 1, 'highway_new': 2}

    for class_name, class_label in class_labels.items():
        class_path = os.path.join(feature_folder_test, class_name)
        for feature_file in os.listdir(class_path):
            if feature_file.endswith(".npy"):  # Load .npy files
                feature_path = os.path.join(class_path, feature_file)
                image_features = np.load(feature_path)
                predicted_class = classify_image(image_features, gmm_params_by_class)

                y_true.append(class_label)
                y_pred.append(predicted_class)

    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)

    print(f"GMM with {n_components} mixtures:")
    print(f"  Accuracy = {accuracy:.4f}")
    print(classification_report(y_true, y_pred))
    print(f"Confusion Matrix:\n{cm}\n")

    return accuracy, report, cm


In [ ]:
feature_folder_train = "/content/drive/MyDrive/group07/features/"
feature_folder_test = "/content/drive/MyDrive/group07/features_test/"
n_components_list = [1, 2, 4]

for n_components in n_components_list:
    accuracy, report, cm = run_experiment(feature_folder_train, feature_folder_test, n_components)

GMM with 1 mixtures:
  Accuracy = 0.6600
              precision    recall  f1-score   support

           0       0.53      0.46      0.49        50
           1       0.77      0.94      0.85        50
           2       0.63      0.58      0.60        50

    accuracy                           0.66       150
   macro avg       0.65      0.66      0.65       150
weighted avg       0.65      0.66      0.65       150

Confusion Matrix:
[[23 11 16]
 [ 2 47  1]
 [18  3 29]]

GMM with 2 mixtures:
  Accuracy = 0.7000
              precision    recall  f1-score   support

           0       0.57      0.62      0.60        50
           1       0.73      0.96      0.83        50
           2       0.87      0.52      0.65        50

    accuracy                           0.70       150
   macro avg       0.72      0.70      0.69       150
weighted avg       0.72      0.70      0.69       150

Confusion Matrix:
[[31 15  4]
 [ 2 48  0]
 [21  3 26]]

GMM with 4 mixtures:
  Accuracy = 0.6933
   

In [ ]:
n_components_list=[8,16,32]
for n_components in n_components_list:
    accuracy, report, cm = run_experiment(feature_folder_train, feature_folder_test, n_components)

GMM with 8 mixtures:
  Accuracy = 0.6733
              precision    recall  f1-score   support

           0       0.56      0.56      0.56        50
           1       0.73      0.94      0.82        50
           2       0.72      0.52      0.60        50

    accuracy                           0.67       150
   macro avg       0.67      0.67      0.66       150
weighted avg       0.67      0.67      0.66       150

Confusion Matrix:
[[28 13  9]
 [ 2 47  1]
 [20  4 26]]



/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
<ipython-input-4-eb9e87e8685a>:4: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariances = np.array([np.cov(X[labels == i].T) for i in range(n_components)])
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

GMM with 16 mixtures:
  Accuracy = 0.6333
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.67      0.98      0.80        50
           2       0.60      0.92      0.72        50

    accuracy                           0.63       150
   macro avg       0.42      0.63      0.51       150
weighted avg       0.42      0.63      0.51       150

Confusion Matrix:
[[ 0 20 30]
 [ 0 49  1]
 [ 0  4 46]]



/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
<ipython-input-4-eb9e87e8685a>:4: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariances = np.array([np.cov(X[labels == i].T) for i in range(n_components)])
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


KeyboardInterrupt: 